STILL UNDER DEVELOPMENT (needs more description and more algorithmic attempts).

We work with a dataset containing 50,000 movie reviews from IMDB, labeled by sentiment (positive/negative).  In addition, there are another 50,000 IMDB reviews provided without any rating labels.  

The reviews are split evenly into train and test sets (25k train and 25k test). The overall distribution of labels is also balanced within the train and test sets (12.5k pos and 12.5k neg).  Our goal is to predict sentiment in the test dataset. 

In [72]:
import os                                # accessing directory of files
import pandas as pd                      # storing the data
from bs4 import BeautifulSoup            # removing HTML tags
import re                                # text processing with regular expressions
from gensim.models import word2vec       # embedding algorithm
import numpy as np                       # arrays and other mathy structures     
from tqdm import tqdm                    # timing algorithms
from gensim import models                # doc2vec implementation
from random import shuffle               # for shuffling reviews
from sklearn.linear_model import LogisticRegression
import nltk.data                         # sentence splitting
from keras.models import Sequential      # deep learning (part 1)
from keras.layers import Dense, Dropout  # deep learning (part 2)
from classes import Doc2VecUtility
from classes import Word2VecUtility
from classes import Word2VecRNNUtility
import matplotlib.pyplot as plt          # plotting
import plotly.plotly as py
import plotly.graph_objs as go
import seaborn as sns
import warnings
import statistics
%matplotlib inline                       

# If you are using Python 3, you will get an error.
# (Pattern is a Python 2 library and fails to install for Python 3.)

The dataset can be downloaded [here](http://ai.stanford.edu/~amaas/data/sentiment/).  We have written code to extract the reviews into Pandas dataframes.

In [2]:
def load_data(directory_name):
    # load dataset from directory to Pandas dataframe
    data = []
    files = [f for f in os.listdir('../../../aclImdb/' + directory_name)]
    for f in files:
        with open('../../../aclImdb/' + directory_name + f, "r", encoding = 'utf-8') as myfile:
            data.append(myfile.read())
    df = pd.DataFrame({'review': data, 'file': files})
    return df

# load training dataset
train_pos = load_data('train/pos/')
train_neg = load_data('train/neg/')

# load test dataset
test_pos = load_data('test/pos/')
test_neg = load_data('test/neg/')

# load unsupervised dataset
unsup = load_data('train/unsup/')

print("\n %d pos train reviews \n %d neg train reviews \n %d pos test reviews \n %d neg test reviews \n %d unsup reviews" \
      % (train_pos.shape[0], train_neg.shape[0], test_pos.shape[0], test_neg.shape[0], unsup.shape[0]))
print("\n TOTAL: %d reviews" % int(train_pos.shape[0] + train_neg.shape[0] + test_pos.shape[0] + test_neg.shape[0] + unsup.shape[0]))


 12500 pos train reviews 
 12500 neg train reviews 
 12500 pos test reviews 
 12500 neg test reviews 
 50000 unsup reviews

 TOTAL: 100000 reviews


`train_pos`, `train_neg`, `test_pos`, `test_neg`, and `unsup` are Pandas dataframes.  They each have two columns, and each row corresponds to a review:
- `file` : name of file that contains review
- `review` : the full text of the review

Each review is cleaned before being fed into a classification algorithm.
- HTML tags are removed through the use of the Beautiful Soup library.
- Punctuation is made consistent through the use of regular expressions.  This code appears below, in the `clean_str` method.
- All words are converted to lowercase.
- Each review is converted into either a list of words or to a list of lists of words.  In the latter, each list of lists corresponds to a sentence.

In [3]:
def clean_str( string ):
    # Function that cleans text using regular expressions
    string = re.sub(r' +', ' ', string)
    string = re.sub(r'\.+', '.', string)
    string = re.sub(r'\.(?! )', '. ', string)    
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'m", " \'m", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " ( ", string) 
    string = re.sub(r"\)", " ) ", string) 
    string = re.sub(r"\?", " ? ", string) 
    string = re.sub(r"\.", " . ", string)
    string = re.sub(r"\-", " - ", string)
    string = re.sub(r"\;", " ; ", string)
    string = re.sub(r"\:", " : ", string)
    string = re.sub(r'\"', ' " ', string)
    string = re.sub(r'\/', ' / ', string)
    return string

We note that there is still some room for improvement.  For instance, 
- Strings like "Sgt. Cutter" currently are broken into two sentences.  We should instead determine how to differentiate between periods that signify the end of an abbreviation and periods that denote the end of a sentence.
- Some writers separate their sentences with commas or line breaks; the algorithm currently absorbs these multiple sentences into an individual sentence.
- Ellipses (...) are currently processed as multiple, empty sentences (which are then discarded).

Before writing this post, I read the Kaggle tutorial [here](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors).  My current processing algorithm borrows from that page, but also adds some meaningful improvements, partially informed by the algorithm [here](https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py).  For instance, 
- We keep the punctuation that ends each sentence (i.e., period vs. exclamation point), whereas punctuation was discarded in the Kaggle tutorial.  
- We do smarter processing of contractions, in a way that understands that "should've" = "should" + "'ve".  In the Kaggle tutorial, "should've" is kept as a single word (contractions are not understood in terms of their composite parts).

# Doc2Vec + (LR, MLP) Implementation

We can now see how well our Doc2Vec embedding performs.  To peek at the code under the hood, please look at the Python file [here](https://github.com/alexisbcook/Pelican_blog_all/blob/master/content/notebooks/classes/Doc2VecUtility.py).

In [4]:
[d2v_train, d2v_test] = Doc2VecUtility.get_embedding()

In [5]:
train_labels = np.append(np.ones(12500), np.zeros(12500))
test_labels = np.append(np.ones(12500), np.zeros(12500))

In [6]:
classifier = LogisticRegression()
classifier.fit(d2v_train, train_labels)
classifier.score(d2v_test, test_labels)

0.89456000000000002

Woohoo!  We can predict sentiment with nearly 90 percent accuracy!  Can we do better?  Let's try out a MLP with one hidden layer.

In [7]:
# Now, we try a multilayer perceptron (with one hidden layer) in Keras !
keras_model = Sequential()
keras_model.add(Dense(200, input_dim=300, init='uniform', activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(1, activation='sigmoid'))
keras_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

keras_model.fit(d2v_train, train_labels, nb_epoch=10, batch_size=20)
loss, accuracy = keras_model.evaluate(d2v_test, test_labels, verbose=0)
print("\n", accuracy)

Epoch 1/10
25000/25000 [==============================] - 2s - loss: 0.2968 - acc: 0.8784     
Epoch 2/10
25000/25000 [==============================] - 2s - loss: 0.2636 - acc: 0.8910     
Epoch 3/10
25000/25000 [==============================] - 2s - loss: 0.2509 - acc: 0.8976     
Epoch 4/10
25000/25000 [==============================] - 2s - loss: 0.2396 - acc: 0.9025     
Epoch 5/10
25000/25000 [==============================] - 3s - loss: 0.2288 - acc: 0.9089     
Epoch 6/10
25000/25000 [==============================] - 3s - loss: 0.2136 - acc: 0.9183     
Epoch 7/10
25000/25000 [==============================] - 3s - loss: 0.2008 - acc: 0.9247     
Epoch 8/10
25000/25000 [==============================] - 3s - loss: 0.1856 - acc: 0.9318     
Epoch 9/10
25000/25000 [==============================] - 3s - loss: 0.1725 - acc: 0.9384     
Epoch 10/10
25000/25000 [==============================] - 3s - loss: 0.1573 - acc: 0.9442     

 0.90504


Hmm, it's a slight improvement, but we haven't done much better.  To improve, our intuition tells us we should try some combination of:
- better string cleaning,
- testing other parameters for Doc2Vec embedding step, and
- constructing deeper neural networks.

However, it seems that Kaggle competitiors were not able to make this work -- that is, we won't be able to do much better with the current plan.  Thus, we will try something else, informed by the techniques [here](https://www.kaggle.com/c/word2vec-nlp-tutorial/forums/t/14966/post-competition-solutions).  Namely, we will train a few more embedding models and then use an ensemble method.

# Word2Vec + (LR, MLP) Implementation

We train a Word2Vec embedding, which we expect will get slightly worse performance than the Doc2Vec model.  To peek at the code under the hood, please look at the Python file [here](https://github.com/alexisbcook/Pelican_blog_all/blob/master/content/notebooks/classes/Word2VecUtility.py).

In [8]:
[w2v_train, w2v_test] = Word2VecUtility.get_embedding()

100%|██████████| 12500/12500 [00:13<00:00, 940.27it/s]


In [9]:
classifier = LogisticRegression()
classifier.fit(w2v_train, train_labels)
classifier.score(w2v_test, test_labels)

0.81396000000000002

In [10]:
# Now, we try a multilayer perceptron (with one hidden layer) in Keras !
keras_model = Sequential()
keras_model.add(Dense(200, input_dim=300, init='uniform', activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(1, activation='sigmoid'))
keras_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

keras_model.fit(w2v_train, train_labels, nb_epoch=10, batch_size=20)
loss, accuracy = keras_model.evaluate(w2v_test, test_labels, verbose=0)
print("\n", accuracy)

Epoch 1/10
25000/25000 [==============================] - 2s - loss: 0.5279 - acc: 0.7516     
Epoch 2/10
25000/25000 [==============================] - 2s - loss: 0.3919 - acc: 0.8302     
Epoch 3/10
25000/25000 [==============================] - 2s - loss: 0.3624 - acc: 0.8451     
Epoch 4/10
25000/25000 [==============================] - 2s - loss: 0.3505 - acc: 0.8504     
Epoch 5/10
25000/25000 [==============================] - 2s - loss: 0.3437 - acc: 0.8543     
Epoch 6/10
25000/25000 [==============================] - 2s - loss: 0.3394 - acc: 0.8571     
Epoch 7/10
25000/25000 [==============================] - 3s - loss: 0.3351 - acc: 0.8574     
Epoch 8/10
25000/25000 [==============================] - 3s - loss: 0.3313 - acc: 0.8607     
Epoch 9/10
25000/25000 [==============================] - 2s - loss: 0.3289 - acc: 0.8612     
Epoch 10/10
25000/25000 [==============================] - 2s - loss: 0.3262 - acc: 0.8627     

 0.86408


# {Doc2Vec, Word2Vec} + (LR, MLP) Implementation

As expected, we the Doc2Vec embedding provides better accuracy than the Word2Vec embedding (both with logistic regression and an MLP).  However, combining the two should result in an increase in performance, as we will see below.

In [11]:
# Combine Doc2Vec and Word2Vec features
d2v_w2v_train = np.append(d2v_train, w2v_train, axis=1)
d2v_w2v_test = np.append(d2v_test, w2v_test, axis=1)

In [12]:
classifier = LogisticRegression()
classifier.fit(d2v_w2v_train, train_labels)
classifier.score(d2v_w2v_test, test_labels)

0.89576

In [13]:
# Now, we try a multilayer perceptron (with one hidden layer) in Keras !
keras_model = Sequential()
keras_model.add(Dense(200, input_dim=600, init='uniform', activation='relu'))
keras_model.add(Dropout(0.5))
keras_model.add(Dense(1, activation='sigmoid'))
keras_model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

keras_model.fit(d2v_w2v_train, train_labels, nb_epoch=10, batch_size=20)
loss, accuracy = keras_model.evaluate(d2v_w2v_test, test_labels, verbose=0)
print("\n", accuracy)

Epoch 1/10
25000/25000 [==============================] - 3s - loss: 0.2946 - acc: 0.8818     
Epoch 2/10
25000/25000 [==============================] - 3s - loss: 0.2617 - acc: 0.8919     
Epoch 3/10
25000/25000 [==============================] - 3s - loss: 0.2458 - acc: 0.8995     
Epoch 4/10
25000/25000 [==============================] - 3s - loss: 0.2344 - acc: 0.9041     
Epoch 5/10
25000/25000 [==============================] - 3s - loss: 0.2229 - acc: 0.9098     
Epoch 6/10
25000/25000 [==============================] - 3s - loss: 0.2071 - acc: 0.9187     
Epoch 7/10
25000/25000 [==============================] - 3s - loss: 0.1949 - acc: 0.9278     
Epoch 8/10
25000/25000 [==============================] - 3s - loss: 0.1787 - acc: 0.9361     
Epoch 9/10
25000/25000 [==============================] - 3s - loss: 0.1644 - acc: 0.9396     
Epoch 10/10
25000/25000 [==============================] - 3s - loss: 0.1514 - acc: 0.9440     

 0.906


Here, unfortunately, the performace is basically the same as when we used Doc2Vec alone.  

# Word2Vec + RNN Implementation

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

[w2vRNN_train, w2vRNN_test] = Word2VecRNNUtility.get_embedding()

100%|██████████| 12500/12500 [00:17<00:00, 701.88it/s]


Our input has variable length!  Keras can't handle this ... so, let's figure out how to deal with it ...

In [26]:
print("The first entry in w2vRNN_train has shape:", w2vRNN_train[0].shape)
print("The second entry in w2vRNN_train has shape:", w2vRNN_train[1].shape)
print("The first entry in w2vRNN_test has shape:", w2vRNN_test[0].shape)
print("The second entry in w2vRNN_test has shape:", w2vRNN_test[1].shape)

The first entry in w2vRNN_train has shape: (156, 300)
The second entry in w2vRNN_train has shape: (475, 300)
The first entry in w2vRNN_test has shape: (168, 300)
The second entry in w2vRNN_test has shape: (393, 300)


In [76]:
review_lengths = [element.shape[0] for element in w2vRNN_train + w2vRNN_test]
print("Minimum review length:", min(review_lengths), "\n")
print("Maximum review length:", max(review_lengths), "\n")
print("Mean review length:", np.mean(review_lengths), "\n")
print("Median review length:", statistics.median(review_lengths), "\n")
# get number of reviews with more than 1000 words
print("Reviews of length >1000:", sum([i>1000 for i in review_lengths]), 
      "(", 100*sum([i>1000 for i in review_lengths])/len(review_lengths), "% ), ",
      "<=1000:", sum([i<=1000 for i in review_lengths]), "\n")

layout = go.Layout(
    xaxis=dict(title='Review Length'),
    yaxis=dict(title='Count'),
    bargap=0.1,
    bargroupgap=0.1
)

data = [go.Histogram(
    x=review_lengths,
    autobinx=True,
    opacity=0.75
)]

warnings.filterwarnings('ignore')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Minimum review length: 6 

Maximum review length: 2582 

Mean review length: 262.14742 

Median review length: 197.0 

Reviews of length >1000: 555 ( 1.11 % ),  <=1000: 49445 



In [17]:
'''
keras_model = Sequential()
keras_model.add(LSTM(100))
keras_model.add(Dense(1, activation='sigmoid'))
keras_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
keras_model.fit(w2vRNN_train, train_labels, nb_epoch=10, batch_size=20)
'''

"\nkeras_model = Sequential()\nkeras_model.add(LSTM(100))\nkeras_model.add(Dense(1, activation='sigmoid'))\nkeras_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\nkeras_model.fit(w2vRNN_train, train_labels, nb_epoch=10, batch_size=20)\n"